In [1]:
%load_ext autoreload
%autoreload 2|

In [2]:
from scipy.special import gammaln
import numpy as np
from tqdm.autonotebook import tqdm
from joblib import Parallel, delayed

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
from childes_mi.information_theory.emi._nij_op_cython import nij_op_cython


In [4]:
from sklearn.metrics.cluster._expected_mutual_info_fast import expected_mutual_information
expected_mutual_information

<function sklearn.metrics.cluster._expected_mutual_info_fast.expected_mutual_information>

In [5]:
from sklearn.metrics import cluster

In [6]:
import numpy as np
import scipy.sparse
from sklearn.metrics.cluster import contingency_matrix
from sklearn.metrics.cluster import adjusted_mutual_info_score

In [28]:
from joblib import parallel_backend

In [12]:
"""
Modified from sklearn
https://github.com/scikit-learn/scikit-learn/blob/b194674c4/sklearn/metrics/cluster/_supervised.py

"""
from sklearn.metrics.cluster.supervised import contingency_matrix
import numpy as np
from sklearn.metrics import (
    mutual_info_score,
    normalized_mutual_info_score,
    adjusted_mutual_info_score,
)



def adjusted_mutual_information(labels_true, labels_pred, n_jobs = -1, emi_method="parallel", use_cython=True,
                               average_method='arithmetic'):
    """Adjusted Mutual Information.
    Adjusted Mutual Information (AMI) is an adjustment of the Mutual
    Information (MI) score to account for chance. It accounts for the fact that
    the MI is generally higher for two clusterings with a larger number of
    clusters, regardless of whether there is actually more information shared.
    For two clusterings :math:`U` and :math:`V`, the AMI is given as::
        AMI(U, V) = [MI(U, V) - E(MI(U, V))] / [avg(H(U), H(V)) - E(MI(U, V))]
    
    """
    n_samples = labels_true.shape[0]
    classes = np.unique(labels_true)
    clusters = np.unique(labels_pred)
    # Special limit cases: no clustering since the data is not split.
    # This is a perfect match hence return 1.0.
    if (classes.shape[0] == clusters.shape[0] == 1 or
            classes.shape[0] == clusters.shape[0] == 0):
        return 1.0
    contingency = contingency_matrix(labels_true, labels_pred, sparse=True)
    contingency = contingency.astype(np.float64,
                                     **_astype_copy_false(contingency))
    # Calculate the MI for the two clusterings
    mi = mutual_info_score(labels_true, labels_pred,
                           contingency=contingency)
    # Calculate the expected value for the mutual information
    if emi_method == "parallel":
        emi = emi_parallel(contingency, n_samples, use_cython = use_cython, n_jobs=n_jobs)
    else:
        emi = expected_mutual_information(contingency, n_samples)
    # Calculate entropy for each labeling
    h_true, h_pred = entropy(labels_true), entropy(labels_pred)
    normalizer = _generalized_average(h_true, h_pred, average_method)
    denominator = normalizer - emi
    # Avoid 0.0 / 0.0 when expectation equals maximum, i.e a perfect match.
    # normalizer should always be >= emi, but because of floating-point
    # representation, sometimes emi is slightly larger. Correct this
    # by preserving the sign.
    if denominator < 0:
        denominator = min(denominator, -np.finfo('float64').eps)
    else:
        denominator = max(denominator, np.finfo('float64').eps)
    ami = (mi - emi) / denominator
    return ami, emi


def entropy(labels):
    """Calculates the entropy for a labeling.
    Parameters
    ----------
    labels : int array, shape = [n_samples]
        The labels
    Notes
    -----
    The logarithm used is the natural logarithm (base-e).
    """
    if len(labels) == 0:
        return 1.0
    label_idx = np.unique(labels, return_inverse=True)[1]
    pi = np.bincount(label_idx).astype(np.float64)
    pi = pi[pi > 0]
    pi_sum = np.sum(pi)
    # log(a / b) should be calculated as log(a) - log(b) for
    # possible loss of precision
    return -np.sum((pi / pi_sum) * (np.log(pi) - np.log(pi_sum)))

def _generalized_average(U, V, average_method="arithmetic"):
    """Return a particular mean of two numbers."""
    if average_method == "min":
        return min(U, V)
    elif average_method == "geometric":
        return np.sqrt(U * V)
    elif average_method == "arithmetic":
        return np.mean([U, V])
    elif average_method == "max":
        return max(U, V)
    else:
        raise ValueError("'average_method' must be 'min', 'geometric', "
                         "'arithmetic', or 'max'")


def _parse_version(version_string):
    version = []
    for x in version_string.split('.'):
        try:
            version.append(int(x))
        except ValueError:
            # x may be of the form dev-1ea1592
            version.append(x)
    return tuple(version)

import scipy
sp_version = _parse_version(scipy.__version__)

def _astype_copy_false(X):
    """Returns the copy=False parameter for
    {ndarray, csr_matrix, csc_matrix}.astype when possible,
    otherwise don't specify
    """
    if sp_version >= (1, 1) or not sp.issparse(X):
        return {'copy': False}
    else:
        return {}

In [26]:
def emi_parallel(contingency, n_samples, use_cython = True, n_jobs=-1, prefer=None):
    """
    EMI without pregenerating lookup table for reduced memory
    https://github.com/clajusch/ClEvaR/blob/master/R/Calculations.R
    """
    
    s1 = np.array(np.sum(contingency, axis=1, dtype="int").flatten()).flatten()
    s2 = np.array(np.sum(contingency, axis=0, dtype="int").flatten()).flatten()
    N = n_samples
    l1 = len(s1)
    l2 = len(s2)

    nijs = np.arange(0, max(np.max(s1), np.max(s2)) + 1, dtype="float")
    nijs[0] = 1
    term1 = nijs / N
    
    log_Nnij = np.log(N * nijs)
    gln_a = gammaln(s1 + 1)
    gln_b = gammaln(s2 + 1)
    gln_Na = gammaln(N - s1 + 1)
    gln_Nb = gammaln(N - s2 + 1)
    gln_N = gammaln(N + 1)
    gln_nij = gammaln(nijs + 1)
    
    if use_cython:
        nij_func = nij_op_cython
    else:
        nij_func = nij_op
    with parallel_backend('multiprocessing'):
        with Parallel(n_jobs=n_jobs, verbose=0, prefer=prefer) as parallel:
            emi = parallel(
                delayed(nij_func)(s1[i], s2, l2, N, term1, nijs, i, gln_a, gln_b, gln_Na, gln_Nb, gln_N, gln_nij, log_Nnij)
                for i in tqdm(range(l1), desc="compute emi", leave=False)
            )
            
    return np.sum(emi)

In [14]:
def nij_op(s1i, s2, l2, N, term1, nijs, i, gln_a, gln_b, gln_Na, gln_Nb, gln_N, gln_nij, log_Nnij):
    emif = 0
    for j in range(l2):
        s2j = s2[j]
        min_nij = np.max([1, s1i - N + s2j])
        max_nij = np.min([s1i, s2j]) + 1
        for nij in np.arange(min_nij, max_nij).astype('int'):
            t1 = term1[nij]
            t2 = log_Nnij[nij] - np.log(s1i * s2j)

            gln = (
                gln_a[i] + 
                gln_b[j] + 
                gln_Na[i] + 
                gln_Nb[j] - 
                gln_N - 
                gln_nij[nij] - 
                gammaln(s1i - nij + 1) - 
                gammaln(s2j - nij + 1) -
                gammaln(N - s1i - s2j + nij + 1)
            )

            t3 = np.exp(gln)
            emi = (t1 * t2 * t3)
            emif += emi
    return emif

In [15]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [16]:
%%cython -a

from libc.math cimport exp, lgamma
from scipy.special import gammaln
import numpy as np
cimport numpy as np
cimport cython


np.import_array()
ctypedef np.float64_t DOUBLE

def nij_op_cython(int s1i, np.ndarray s2, int l2, int N, np.ndarray term1, 
                  np.ndarray nijs, int i, np.ndarray gln_a, np.ndarray gln_b, 
                  np.ndarray gln_Na, np.ndarray gln_Nb, float gln_N, np.ndarray gln_nij,
                  np.ndarray log_Nnij ######### create ###########
                 ):
    cdef DOUBLE emi, log_aibj, term3, gln, term2
    cdef int min_nij, max_nij, s2j
    emi = 0
    cdef Py_ssize_t j, nij
    for j in range(l2):
        s2j = s2[j]
        min_nij = max([1, s1i + s2j - N])
        max_nij = min([s1i, s2j]) + 1
        
        log_aibj = np.log(s1i * s2j)
        for nij in range(min_nij, max_nij):
            term2 = log_Nnij[nij] - log_aibj
             
            gln = (
                    gln_a[i]
                    + gln_b[j]
                    + gln_Na[i]
                    + gln_Nb[j]
                    - gln_N
                    - gln_nij[nij]
                    - lgamma(s1i - nij + 1)
                    - lgamma(s2j - nij + 1)
                    - lgamma(N - s1i - s2j + nij + 1)
            )
               
            term3 = exp(gln)
            
            emi += (term1[nij] * term2 * term3)
        
    return emi

In [17]:
# 2.07
C = scipy.sparse.load_npz('shuffled_contingency.npz')
n_samples = C.sum()
n_samples, C.shape

(60242.0, (3998, 3998))

In [19]:
contingency = C.astype(np.float64,**_astype_copy_false(C))

In [29]:
emi = emi_parallel(contingency, n_samples, use_cython = True, n_jobs=-1)